# The North Remembers Project

In [1]:
%load_ext autoreload
%autoreload 2

### 取舍

数据存储用 github, 或者 bitbucket


提供导出为 rss, epub, pdf, github 自带检查历史的功能

不追求实效性



外链图片不作为重点
外链图片和视频链接, url 链接, 一并修复, 换行字符串等也一并处理, 七牛云?
md 中的 Latex 和 代码检测也在这里修复, pygment

### 调用关系





#### Collector

处理用户发出的命令

可以是 zhihu answer url, 问题页, 收藏夹页, 需要带有过滤条件
用户可能在过滤条件后再做修补删减

需要转为具体的爬虫任务, 并创建文件夹
单页收集到特定的文件夹, 否则收集到一个自动命名的文件夹

在文件夹下生成task

Resources
global_config
task_config (scanlist)


Collector.append(url, options)



#### Watcher



任务调度, 监视文件夹, 根据 task.yaml 抓取页面

task.yaml 也记录历史

task_type: single_page, 
update_batch: 10


track_new: true


通过扫描已有的待抓取任务, 在手动触发, 达到特定时间后, 分派抓取任务

    Watcher
        .watch()
        .remember()
        .status()
        .load_config_yaml
        .save_config_yaml
        .add_task add_tasks



Task

被调用抓取单一页面内容, 或者抓取一个 url 列表
抓取单一页面内容将存为 md, 不存 html

    Task
        ZhihuAnswerPageTask
        ZhihuAnswerIndexTask
        ZhihuColumnPageTask
        ZhihuColumnIndexTask

Task 会使用 Page.save / load

    .last_page()
    .fetch()
    .save() 传递一些参数给 page 如 watch_date
    .to_yaml_text()
    
    .priority
    .is_page_type
    .should_fetch
    .content_is_changed



ZhihuColumnPage
ZhihuColumnLister


Fetcher.request(url) => dict()? Page()

Page.write to / load from disk

PageTask.run => Fetcher.request(url) => New Page
PageTask.update => New Page.write
IndexTask.run => Fetcher.request(url) => [New Tasks]


Watcher -> PageTask -> Page (tmpl, fetcherAPI)
                            ZhihuColumnPage
                            ZhihuAnswerPage
                            WeixinArticlePage


Watcher -> IndexTask -> append more tasks
                        ZhihuColumnIndex
    





#### Fetcher

Page

页面内容分为 元数据 + 文章主体 + 评论, 评论先放放


Page.load()  返回一个读盘的页面

以md文件为单位

    ZhihuColumnPage
        .load(path)
        .write(path)
        .compare()

    attr
        task = ForeignKeyField(Task, related_name='task')
        watch_date = DateTimeField()
        title = CharField(null=True)
        author = CharField(null=True)
        content = TextField(null=True)
        metadata = TextField(null=True)
        topic = TextField(null=True)
        question = TextField(null=True)
        comment = TextField(null=True)





#### Cleaner

处理, 清理文本的规则



Exporter

根据 Template 转换为 rss, epub, pdf, 以及 github diff 预览

图片转存也算在 Template 里

In [ ]:
描述

以更加简洁清晰的排版阅读知乎. (这个需求有简悦)

用户指定一个知乎网站上的问题, zhihumark 抓取相应的回答内容, 以 markdown 文档的形式提供给用户阅读和下载. (提供 md 下载没用)





功能

可以自由定制想抓取的内容, 这包括: 抓取指定答主/收藏夹名/问题/话题下的回答, 抓取赞同数量超过N的回答.
可以设置黑名单, 不抓取特定的问题或者答主.
抓取评论, 以会话的形式组织起来, 让上下文相关的评论出现在一起, 尽量滤除没有价值的评论.
将收集到的一系列文档保存为epub或者pdf, 或者批量下载 markdown 文档, 以便离线阅读.
制作成 feed 以便放入 rss 阅读器, 获取实时更新.


观念

我们也不显示其他人搜索了什么关键字, 获取了什么页面, 下载了哪些电子书, 看到什么取决于你

我们相信评论是回答(或文章)可以分割的一部分, 许多评论很水, 有些很有价值.

相信评论区是对回答的重要补充, 也有随回答保留的必要, 但需处理评论区良莠不齐, 引用回复混乱的问题.



页面设计

单页面网站应用, 基本上类似搜索引擎的风格和排版.

输入框 - 接受输入ID或URL 答主/收藏夹名/问题名/话题名

选项 - 保留最前的n个回答?
选项 - 赞同数至少是?
选项 - 保留评论? 评论对话中至少有多少 likes
屏蔽 - 不感兴趣的话题, 问题, 答主
返回 - markdown文件列表

批量下载按钮

In [ ]:
def test_fetch_history():
  # 美国是不是正在为瓦解中国做准备？ - 张俊麒的回答
  # url = 'https://www.zhihu.com/question/40622033/answer/88139460'
  
  # 有哪些令人拍案叫绝的临场反应？ - 大盗贼霍老爷的回答
  url = 'https://www.zhihu.com/question/29125576/answer/54187388'
  
  # 机器人教育这种不考试、以娱乐为主的教育对于中小学生及幼儿的意义何在？ - 冷哲的回答"
  url = 'https://www.zhihu.com/question/26999365/answer/93421419'
  
  
  # 2016 年，中国房地产泡沫是否会在一两年内破灭，从而引发金融危机？ - 小马的回答"
  url = 'https://www.zhihu.com/question/40910547/answer/123021503'
  
  # Smart Beta 投资方法 - 陈颖的专栏 量化哥"
  url = 'https://zhuanlan.zhihu.com/p/21936302'
  
  
  # 美国南北战争的真正原因是什么？ - talich的回答"
  url = 'https://www.zhihu.com/question/19679278/answer/15122151'
  # query = (Page.select(Page, Task)
  #          .join(Task)
  #          .where((Task.page_type == 'zhihu_article') & (Page.title.contains('最前沿')))
  #          .group_by(Page.task)
  #          .having(Page.watch_date == fn.MAX(Page.watch_date))
  #          .limit(9999))
  t = Task.select().where(Task.url == url).get()
  
  
  contents = []
  for page in t.pages:

    print(page.version)
    print(page.title)
    print(page.topic)
    contents.append(page.content)
#     print(page.content)
  return contents
contents = test_fetch_history()

In [ ]:
def split_text(text):
  col = []
  for sent in re.findall('.*?[！|，|。|？|\n|；|~|～|：|\)|\]]', text):
    col.append(sent)
  
  return col

print(split_text(a))

In [ ]:
print(len(contents))
# c1, c2, c3, c4, *_, c7 = contents
from difflib import unified_diff
def compare_text(t1, t2, prefix=''):
  changes = [l for l in unified_diff(t1, t2)]

  return changes


import re
for i, (a, b) in enumerate(zip(contents[:-1], contents[1:])):
#   if i == 0:
#     continue
  if (a!=b):
    print('a!=b on', i, i+1)
    
    pat = re.compile('\S')
    for d in unified_diff(split_text(a), split_text(b), n=2, lineterm=''):
      print(d)
  
  print('\n\n\n\n\n\n')
#   if i >= 5:
#     break
  
# compare_text(c1, c2)
# compare_text(c2, c3)

In [ ]:
'iesytwe'.split('?')

In [ ]:

# from unidiff import PatchSet
# diff = urllib2.urlopen('https://github.com/matiasb/python-unidiff/pull/3.diff')
# encoding = diff.headers.getparam('charset')
# patch = PatchSet(diff, encoding=encoding)


### 原 models test exec




In [ ]:

'''
####### #######  ###### #######
   ##   ##      ##         ##
   ##   ######   #####     ##
   ##   ##           ##    ##
   ##   ####### ######     ##
'''

def test_new_task():
  # url = 'https://www.zhihu.com/question/30595784/answer/49194862'
  # 如何看待许知远在青年领袖颁奖典礼上愤怒「砸场」？
  url = 'https://www.zhihu.com/question/22316395/answer/100909780'
  url = 'https://www.zhihu.com/question/47220155/answer/118154455'
  url = 'https://www.zhihu.com/question/49962599/answer/118716273'
  url = 'https://zhuanlan.zhihu.com/p/19837940'
  url = 'https://zhuanlan.zhihu.com/p/20639779'
  url = 'https://zhuanlan.zhihu.com/p/20153329'
  url = 'https://zhuanlan.zhihu.com/p/21281864'
  url = 'https://zhuanlan.zhihu.com/p/19964142'
  task = Task.add(url=url)
  print(task)
  # task.watch()


def test_readd_task():
  url = 'http://www.zhihu.com/question/22513722/answer/21967185' # 火车票涨价
  task = Task.add(url=url)
  task = Task.add(url=url)
  url = 'https://www.zhihu.com/question/30957313/answer/50266448' # 古典音乐
  task = Task.add(url=url)
  url = 'https://www.zhihu.com/question/40056948/answer/110794550' # 四万亿
  task = Task.add(url=url)

def test_seed_add_tasks():
  urls = '''
    https://www.zhihu.com/question/40305228/answer/86179116
    https://www.zhihu.com/question/36466762/answer/85475145
    https://www.zhihu.com/question/33246348/answer/86919689
    https://www.zhihu.com/question/39906815/answer/88534869

    https://www.zhihu.com/question/40700155/answer/89002644
    https://www.zhihu.com/question/36380091/answer/84690117
    https://www.zhihu.com/question/33246348/answer/86919689
    https://www.zhihu.com/question/35254746/answer/90252213
    https://www.zhihu.com/question/23618517/answer/89823915

    https://www.zhihu.com/question/40677000/answer/87886574

    https://www.zhihu.com/question/41373242/answer/91417985
    https://www.zhihu.com/question/47275087/answer/106335325
    https://www.zhihu.com/question/47275087/answer/106335325 买不起房是房价太高还是工资太低？
    https://www.zhihu.com/question/36129534/answer/91921682  印度经济会在本世纪追上中国吗？
    https://www.zhihu.com/question/22513722/answer/21967185  火车票涨价是否能解决春运问题？
    https://www.zhihu.com/question/27820755/answer/107267228 裸辞后怎样解释以获工作机会？
  '''
  for url in datalines(urls):
    url = url.split(' ')[0]
    task = Task.add(url=url)
    task | puts()



def test_one_watch():
  task = Task.select().order_by(-Task.id).get()
  task | puts()
  task.watch()

def test_another_watch():
  url = 'http://www.zhihu.com/question/22513722/answer/21967185' # 火车票涨价
  task = Task.select().where(Task.url == url).get()
  task | puts()
  task.watch()


def test_hot_answer():
  url = 'http://www.zhihu.com/question/39288165/answer/110207560'
  url = 'https://www.zhihu.com/question/50763374/answer/122822226'
  url = 'https://www.zhihu.com/question/40910547/answer/123021503'
  url = 'https://zhuanlan.zhihu.com/p/21478575'
  url = 'https://www.zhihu.com/question/40103788/answer/124499334'
  task = Task.add(url=url)
  task.watch()


  task.last_page.to_local_file(folder='test', fetch_images=False)



def test_watch_all():
  Task.multiple_watch(sleep_seconds=10, limit=4)


def test_report():
  Task.report()



def test_to_local_file():
  # page = Page.select().order_by(-Page.id).get()

  # page = Page.select(Page.topic).distinct().where(Page.topic.contains('房')).limit(5)
  # q = Page.select(Page.id).distinct()
  # for p in q:
  #   print(p)
  query = (Page.select(Page, Task)
           .join(Task)
           .where(Page.author == 'chenqin')  # .where(Page.topic.contains('建筑'))
           .group_by(Page.task)
           .having(Page.watch_date == fn.MAX(Page.watch_date))
           .limit(8800))
  for page in query:
    log(page.title)
    # log(page.metadata)
    # page.to_local_file(folder='chen', fetch_images=False)
# test_to_local_file()

def test_tools():
  import pylon
  pylon.generate_figlet('task', fonts=['space_op'])
  pylon.generate_figlet('page', fonts=['space_op'])
  pylon.generate_figlet('test', fonts=['space_op'])


def test_load_json():
  import json

  print(json.loads(open('mockup_topic_answers.json', encoding='utf-8').read()))


def test_banned_modes():
  url = 'https://www.zhihu.com/question/40679967/answer/88310495'
  # 政府推出开放小区政策的真正目的是什么？ 2201 孟德尔 回答建议修改：政治敏感
  pass


def test_query_task():
  s = 'https://www.zhihu.com/question/48737226/answer/113036453'
  s = 'https://www.zhihu.com/question/47220155/answer/118154455'
  s = 'https://www.zhihu.com/question/49545583/answer/116529877'

  t = Task.select().where(Task.url == s)
  t = t.get()
  log(t.title)
  log(t.id)






def test_explore():
  # Tweet.select(fn.COUNT(Tweet.id)).where(Tweet.user == User.id)
  query = (Task
           .select(Task, fn.COUNT(Page.id).alias('fetched_count'))
           .join(Page)
           .group_by(Task.title)
           .limit(50)
           .offset(200)
           .order_by(fn.COUNT(Page.id).desc()))

  for task in query:
    log(task.title + ' : ' + str(task.fetched_count) + '  task_id ' + str(task.id))




def test_explore_watching_results_diff():

  s = '''
  # 美国是不是正在为瓦解中国做准备？ - 张俊麒的回答 : 16  task_id 46
  # 为什么很少看到患者砍莆田系医生的报道？ - 玄不救非氪不改命的回答 : 3  task_id 196
  # 为什么很难证伪马克思主义理论？ - 玄不救非氪不改命的回答 : 3  task_id 360
  # 为什么快速浏览一段内容的时候，很容易看到自己感兴趣的部分？ - 采铜的回答 : 3  task_id 742
  # 为什么拿广州恒大淘宝队与中国国家男子足球队做对比？ - 玄不救非氪不改命的回答 : 3  task_id 492
  # 如何看待里约奥运陈欣怡药检呈阳性反应？ - 玄不救非氪不改命的回答 : 3  task_id 2393
  # 为什么厌恶「国粉」的知乎用户远多于厌恶「毛粉」的？ - chenqin的 : 3  task_id 3313
  # 2016 年中国的经济状况很差吗？真实状况是怎样的？ - 垒起善城堡的积木 : 3 task_id 2387
  # 如何看待2016年7月人民币贷款增幅里9.8成为房贷？ - 匿名用户的回答 : 3 task_id 2386




  # 怎样评价「游戏不汉化就差评」的行为？ - cOMMANDO的回答 : 9  task_id 4471
  # 既然有报道说人类的基因片段只占DNA序列总长的不到10%，那么这几个问题怎么解答？ - Mandelbrot的回答 : 9  task_id 676
  # 智商低的人真的不适合玩需要动脑子的游戏么？ - 匿名用户的回答 : 9  task_id 3461
  # 暴雪，Valve，拳头，谁更厉害？ - cOMMANDO的回答 : 9  task_id 2597
  # 有一个稀有的姓是一种怎样的体验？ - 冷哲的回答 : 9  task_id 1041
  # 有什么影视作品是当时演员名气不大，现在看来是全明星阵容出演？ - 玄不救非氪不改命的回答 : 9  task_id 484
  # 有关白龙尾岛的历史，哪些是有据可查的？ - 书生的回答 : 9  task_id 712
  # 有哪些「智商税」？ - 谢熊猫君的回答 : 9  task_id 2406
  # 有哪些令人拍案叫绝的临场反应？ - 大盗贼霍老爷的回答 : 9  task_id 4107
  # 有哪些可怕的故事？ - 大盗贼霍老爷的回答 : 9  task_id 4137
  # 有哪些长得比较逆天的动物？ - Mandelbrot的回答 : 9  task_id 648
  # 有文化有多可怕？ - 寺主人的回答 : 9  task_id 5787
  # 机器人教育这种不考试、以娱乐为主的教育对于中小学生及幼儿的意义何在？ - 冷哲的回答 : 9  task_id 1015
  # 毛花三年打败蒋然后走三十年弯路的目的，都是为后三十年的改革开放走资、大国崛起做铺垫扫平道路的么？ - 书生的回答 : 9  task_id 99
  # 水旱蝗汤中的汤指的到底是谁？ - 书生的回答 : 9  task_id 104
  # 河南的地理位置那么好，为什么经济落后？ - 大盗贼霍老爷的回答 : 9  task_id 4105
  # 為什麼蒋介石被称为运输大队长？求详? - 书生的回答 : 9  task_id 701
  # 玩《狼人杀》时你有什么屡试不爽的秘技诀窍？ - 汪诩文的回答 : 9  task_id 3526

  现在网络上很多人黑一些伟人，比如说周半期，黑鲁迅。他们是什么心态？ - 书生的回答 : 9  task_id 97
  看美剧、英剧学英语有什么有效的方法吗？ - 采铜的回答 : 9  task_id 787
  章鱼的智商到底有多高，为什么有人说它们的智商可以统治世界? - Mandelbrot的回答 : 9  task_id 588
  类似 AlphaGo 的人工智能在游戏王、万智牌等卡牌游戏中胜率如何？ - 莫名的回答 : 9  task_id 3724
  给 59 分强行不给过的老师是一种怎么样的存在？ - chenqin的回答 : 9  task_id 3317
  网络上有哪些广为流传的「历史真相」其实是谣言？ - 马前卒的回答 : 9  task_id 5089
  美国南北战争的真正原因是什么？ - talich的回答 : 9  task_id 2571
  美国发动伊拉克战争的核心原因到底是什么？ - 冷哲的回答 : 9  task_id 1332
  美国最高法院大法官 Scalia 的去世将会带来怎样的影响？ - talich的回答 : 9  task_id 4423
  美国有人在开车在路上故意把川普的竞选宣传牌碾倒，如何评价这种因为不同政见而破坏对方财物的行为？ - talich的回答 : 9  task_id 4412
  装逼成功是怎样一种体验？ - 大盗贼霍老爷的回答 : 9  task_id 4048
  谁最应该被印在人民币上面？ - 蜂鸟的回答 : 9  task_id 2274
  豆瓣的核心用户都有什么特点？ - 十一点半的回答 : 9  task_id 84
  赌场有哪些看似不起眼，实则心机颇深的设计？ - 第一喵的回答 : 9  task_id 3158
  赌场有哪些看似不起眼，实则心机颇深的设计？ - 肥肥猫的回答 : 9  task_id 3151
  雷锋是个什么样的人，怎么客观评价雷锋？ - 书生的回答 : 9  task_id 96
  鲁迅和秋瑾的关系好吗？ - 书生的回答 : 9  task_id 91
  1927 年蒋介石为什么要清党？ - 冷哲的回答 : 8  task_id 1415
  1949年以后的中国本土设计的建筑中，哪些能称得上是有思想的好建筑？ - Chilly的回答 : 8  task_id 3104
  2015 年初，中国制造业形势有多严峻？ - 稻可道 稻子的稻的回答 : 8  task_id 2239
  2016 年，中国房地产泡沫是否会在一两年内破灭，从而引发金融危机？ - Bee Mad的回答 : 8  task_id 2104
  2016 年，中国房地产泡沫是否会在一两年内破灭，从而引发金融危机？ - 君临的回答 : 8  task_id 2205
  2016 年，中国房地产泡沫是否会在一两年内破灭，从而引发金融危机？ - 小马的回答 : 8  task_id 5752
  ISIS 是一个什么样的组织？它的资金是哪来的？ - 罗晓川的回答 : 8  task_id 39
  Lambda 表达式有何用处？如何使用？ - 涛吴的回答 : 8  task_id 2008
  Signal Weighting---基于因子IC的因子权重优化模型 - 陈颖的专栏 量化哥 : 8  task_id 6007
  Smart Beta 投资方法 - 陈颖的专栏 量化哥 : 8  task_id 6017
  ofo 获滴滴数千万美元C轮投资，然后呢？ - 曲凯的专栏 创投方法论 : 8  task_id 5656
  《文明 6 》中的背景音乐都有什么来历？ - PenguinKing的回答 : 8  task_id 6250
  《权力的游戏》你觉得最可怜的人是谁？ - 苏鲁的回答 : 8  task_id 4576
  《蒋介石日记》和《毛泽东选集》差距有多大？ - 马前卒的回答 : 8  task_id 5355
  「心灵鸡汤」式的文章错在哪？ - 赵皓阳的回答 : 8  task_id 584
  '''

  for line in datalines(s):
    task_id = int(line.split('task_id')[-1][1:])
    # log(task_id)
    task = Task.select().where(Task.id == task_id).get()
    log(task)
    # log(task.pages)
    # log(task.last_page)
    contents = [fix_in_compare(p.content) for p in task.pages]
    # questions = [fix_in_compare(p.question) for p in task.pages]
    titles = [p.title for p in task.pages]

    # metas = [p.metadata for p in task.pages]
    # for meta in metas:
    #   log(meta)
    # compare_text_sequence(titles, label='titles')
    # compare_text_sequence(questions, label='questions')
    compare_text_sequence(contents, label='contents')

    log('\n\n\n')



def fix_in_compare(text):
  import re
  img_reg = r'\n*\!\[\]\((https?://pic[^()]+?(\.jpg|\.png|\.gif))\)\n*'
  pattern_img_start_inline = re.compile(img_reg)
  def replace_img_start_inline(mat):
    # 保证生成的 *.md 图片在新的一行
    s = mat.group(0)
    while not s.startswith('\n\n'):
      s = '\n' + s
    while not s.endswith('\n\n'):
      s = s + '\n'
    return s

  text = pattern_img_start_inline.sub(replace_img_start_inline, text)

  pattern_img_https = re.compile(r'http://pic(\d)\.zhimg\.com')
  text = pattern_img_https.sub(r'https://pic\1.zhimg.com', text)
  return text


def compare_text(t1, t2, prefix=''):
  import difflib
  changes = [l for l in difflib.ndiff(t1.split('\n'), t2.split('\n')) if l.startswith(('+ ', '- '))]
  for change in changes:
    log(prefix + change)
  return changes


def compare_text_sequence(texts, label=''):
  from pylon import dedupe
  from pylon import windows

  texts = list(dedupe(texts))
  if len(texts) > 1:
    log('detect changed {}'.format(label))
    for t1, t2 in windows(texts, length=2, overlap=1):
      compare_text(t1, t2, prefix='  ')
  else:
    log('nothing changed {}'.format(label))



def test_explore_voteup_thanks():
  '''感谢赞同比跟文章质量没啥关系'''
  query = (Page.select(Page, Task)
           .join(Task)
           .where((Task.page_type == 'zhihu_answer'))
           .group_by(Page.task)
           .having(Page.watch_date == fn.MAX(Page.watch_date))
           .limit(50)
           .order_by(fn.Random())
           )

  def thanks_voteup_ratio(page):
    thanks = int(page.metadata.split('thanks: ')[1].split(' ')[0])
    voteup = int(page.metadata.split('voteup: ')[1].split(' ')[0])
    return round(thanks / voteup, 3)

  # for page in query:
  #   log(page.title)

  pages = sorted(query, key=thanks_voteup_ratio)
  for page in pages:
    log(page.title)
    log(repr(page.content[:500]))
    log(thanks_voteup_ratio(page))
    log('-----------------\n\n\n')


### Test

In [ ]:
from collector import Collector

col = Collector.load(file='config.yaml')

In [ ]:
col.all()

In [ ]:
col.add('https://www.zhihu.com/question/49545583/answer/116529877')

In [ ]:
col.scan()

In [ ]:
collector.add(weixin_page_url)
collector.add(zhihu_column_url)
collector.add(zhihu_anwser_id=xxx)
collector.add(zhihu_author_id=xxx)
collector.add(column_id=xxx)

collector.save(file=xxx)
...
collector.rescan()
collector.save(file=xxx)
collector = Collector.load(file=xxx.yaml)

### Test Watcher

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from watcher import Watcher
# w = Watcher(r'D:\Coding\TheNorthRemembers\test\旗舰评论——战略航空军元帅的旗舰')
# w = Watcher(r'D:\Coding\TheNorthRemembers\test\知乎 - 温酒的回答')
# w = Watcher(r'D:\Coding\TheNorthRemembers\test\经济')
w = Watcher(r'D:\Coding\TheNorthRemembers\test\人文主义谈话录')
# w = Watcher(r'D:\Coding\TheNorthRemembers\test\HelloFlask')


loaded config_data option lister: 
{'enabled': True,
 'limit': 999,
 'max_cycle': '15day',
 'min_cycle': '1hour',
 'weight': 0.5,
 'zhihu_min_voteup': 1}
loaded config_data option page: 
{'enabled': True, 'max_cycle': '1day', 'min_cycle': '1hour', 'weight': 0.5}
loaded config_data lister tasks 3
loaded config_data page tasks 243
Watcher.add_tasks result status , Counter({'added': 3})
Watcher.add_tasks result status , Counter({'added': 243})


In [ ]:
t = w.tasks
t[0]
t[0].option


In [65]:
w.watch()

Watcher.watch should fetch 3 lister_type tasks

Task.run lister request: <Task #2501068430248> https://zhuanlan.zhihu.com/humanism1-100 (ver. 1)
    人文主义谈话录
    task add: an hour ago, last watch: an hour ago, last change an hour ago, next watch: 7 minutes ago 
request_ZhihuColumnLister column_id, humanism1-100
extract new task {'url': 'https://zhuanlan.zhihu.com/p/55784085', 'tip': '122-exurb1a：然后我们会好吗？ - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/38940985', 'tip': '121-Rare Earth：历史断面五则 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/36528562', 'tip': '120-理查德.道金斯/尼尔.德格拉斯.泰森：对谈答疑录 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/35386407', 'tip': '119-Margarita Georgitseas：瓦肯人、纳粹与好莱坞的法西斯 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/35190280', 'tip': '118-加布.纽维尔：游戏与商业 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/34799805', 'tip': '117-阿兰.德波顿：无神论者应当向宗教学习什么 - wnkk'}
extract new task {'url': 'https://z

extract new task {'url': 'https://zhuanlan.zhihu.com/p/24638529', 'tip': '42-Sam Harris：我为什么反对使用“无神论者”这个词 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/24638477', 'tip': '41-亚当.萨维奇：手工制造十诫 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/24638410', 'tip': '40-加里.卡斯帕罗夫：成功的因素 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/24638346', 'tip': '39-奈尔·德葛拉司·泰森：怀疑论拾零 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/24638270', 'tip': '38-阿兰.德.波顿：论地位焦虑 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/24638228', 'tip': '37-Simon Sinek：从“为什么”开始 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/24628500', 'tip': '36-阿兰.德.波顿：论悲观主义的益处 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/24628402', 'tip': '35-Ayaan Hirsi Ali：谁在以自由之名侵犯自由？ - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/24626081', 'tip': '34-希拉里.克林顿：论LGBT群体的权益保护 - wnkk'}
extract new task {'url': 'https://zhuanlan.zhihu.

extract new task {'url': 'https://zhuanlan.zhihu.com/p/38952393', 'tip': '进击的巨人如果走奇幻路线会是怎样的？ - 艾尔文团长'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/37929507', 'tip': '银河中的伦理思考：一人决定牺牲民众顾全“大局”是否合理？ - 艾尔文团长'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/37862774', 'tip': '巨人106话：阿尔敏与艾伦的分歧，巨人会走向反战还是军国主义？ - 艾尔文团长'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/37515285', 'tip': '民主和专制的效率问题，专制就一定效率高，民主就一定低吗 - 艾尔文团长'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/37419679', 'tip': '银河英雄传中的战术思想（一）：各个击破与新月包抄 - 艾尔文团长'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/37338558', 'tip': '刺客信条叛变的思考：对盲目追求自由的质疑，自由与秩序该如何取舍 - 艾尔文团长'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/36404175', 'tip': '巨人105话：曾经的单纯与美好，一去不复返了 - 艾尔文团长'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/36324780', 'tip': '国士无双：管仲是如何运用经济学让齐国成为春秋五霸的？ - 艾尔文团长'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/36157857', 'tip': '东京喰种re（二）：“人类宠物”与奴隶贸易，喰种的黑暗物语 - 艾尔文团长'}

extract new task {'url': 'https://zhuanlan.zhihu.com/p/28014286', 'tip': '（十）阶级论 - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/27307969', 'tip': '（十四）应试补遗（中） - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/27305149', 'tip': '（十三）应试补遗（上） - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/27233707', 'tip': '（九）资本论（下） - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/26174617', 'tip': '（八）异化论 - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/26156420', 'tip': '自省文其二 - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/26151013', 'tip': '自省文其一 - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/26147974', 'tip': '（七）资本论（上） - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/26064765', 'tip': '（六）矛盾论（下） - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/26052254', 'tip': '（五）真理 - JoJo王颀'}
extract new task {'url': 'https://zhuanlan.zhihu.com/p/25916602'

Task.run fetch page done
warning! already exist D:\Coding\TheNorthRemembers\test\人文主义谈话录/111-Atul Gewande：怎样合理地迎接死亡 - wnkk的专栏 人文主义谈话录.md
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/111-Atul Gewande：怎样合理地迎接死亡 - wnkk的专栏 人文主义谈话录.md done
Watcher.watch page task done: <Task #2501067958592>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Task.run page request: <Task #2501067959992> https://zhuanlan.zhihu.com/p/32514551 (ver. 1)
    110-Extra Sci Fi：六评《弗兰肯斯坦》 - wnkk
    task add: an hour ago, last watch: an hour ago, last change an hour ago, next watch: 7 minutes ago 
Task.run fetch page done
warning! already exist D:\Coding\TheNorthRemembers\test\人文主义谈话录/110-Extra Sci Fi：六评《弗兰肯斯坦》 - wnkk的专栏 人文主义谈话录.md
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/110-Extra Sci Fi：六评《弗兰肯斯坦》 - wnkk的专栏 人文主义谈话录.md done
Watcher.watch page task done: <Task #2501067959992>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Watcher.remember add

...

Task.run page request: <Task #2501068119288> https://zhuanlan.zhihu.com/p/24638228 (ver. 1)
    37-Simon Sinek：从“为什么”开始 - wnkk
    task add: an hour ago, last watch: an hour ago, last change an hour ago, next watch: 7 minutes ago 
Task.run fetch page done
warning! already exist D:\Coding\TheNorthRemembers\test\人文主义谈话录/37-Simon Sinek：从“为什么”开始 - wnkk的专栏 人文主义谈话录.md
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/37-Simon Sinek：从“为什么”开始 - wnkk的专栏 人文主义谈话录.md done
Watcher.watch page task done: <Task #2501068119288>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Task.run page request: <Task #2501068118560> https://zhuanlan.zhihu.com/p/24628500 (ver. 1)
    36-阿兰.德.波顿：论悲观主义的益处 - wnkk
    task add: an hour ago, last watch: an hour ago, last change an hour ago, next watch: 7 minutes ago 
Task.run fetch page done
warning! already exist D:\Coding\TheNorthRemembers\test\人文主义谈话录/36-阿兰.德.波顿：论悲观主义的益处 - wnkk的专栏 人文主义谈话录.md
write D:\Coding\TheNorthRemembers\test\人文主义谈话

Task.run page request: <Task #2501068144200> https://zhuanlan.zhihu.com/p/24618391 (ver. 1)
    21-David Foster Wallace：这就是水 - wnkk
    task add: an hour ago, last watch: an hour ago, last change an hour ago, next watch: 7 minutes ago 
Task.run fetch page done
warning! already exist D:\Coding\TheNorthRemembers\test\人文主义谈话录/21-David Foster Wallace：这就是水 - wnkk的专栏 人文主义谈话录.md
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/21-David Foster Wallace：这就是水 - wnkk的专栏 人文主义谈话录.md done
Watcher.watch page task done: <Task #2501068144200>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Task.run page request: <Task #2501068144144> https://zhuanlan.zhihu.com/p/24618271 (ver. 1)
    20-Sam Harris 论道德问题的科学解答 - wnkk
    task add: an hour ago, last watch: an hour ago, last change an hour ago, next watch: 7 minutes ago 
Task.run fetch page done
warning! already exist D:\Coding\TheNorthRemembers\test\人文主义谈话录/20-Sam Harris论道德问题的科学解答 - wnkk的专栏 人文主义谈话录.md
write D:\Coding\TheNo

Task.run page request: <Task #2501068180336> https://zhuanlan.zhihu.com/p/23633905 (ver. 1)
    6-杰克.格里森：我为什么讨厌名人文化 - wnkk
    task add: an hour ago, last watch: an hour ago, last change an hour ago, next watch: 7 minutes ago 
Task.run fetch page done
warning! already exist D:\Coding\TheNorthRemembers\test\人文主义谈话录/6-杰克.格里森：我为什么讨厌名人文化 - wnkk的专栏 人文主义谈话录.md
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/6-杰克.格里森：我为什么讨厌名人文化 - wnkk的专栏 人文主义谈话录.md done
Watcher.watch page task done: <Task #2501068180336>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Watcher.remember add and commit save pages 108
Task.run page request: <Task #2501068180672> https://zhuanlan.zhihu.com/p/23568785 (ver. 1)
    4-凯文.史派西：论电视行业与创意人才 - wnkk
    task add: an hour ago, last watch: an hour ago, last change an hour ago, next watch: 7 minutes ago 
Task.run fetch page done
warning! already exist D:\Coding\TheNorthRemembers\test\人文主义谈话录/4-凯文.史派西：论电视行业与创意人才 - wnkk的专栏 人文主义谈话录.md
write D:\C

Task.run page request: <Task #2501068681400> https://zhuanlan.zhihu.com/p/33735366 (ver. 0)
    韶山斩龙决第叁式·昊天塔 - JoJo王颀
    task add: an hour ago, last watch: None, last change None, next watch: an hour ago 
Task.run fetch page done
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/韶山斩龙决第叁式·昊天塔 - JoJo王颀的专栏 马克思的记事本.md done
Watcher.watch page task done: <Task #2501068681400>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Task.run page request: <Task #2501068684536> https://zhuanlan.zhihu.com/p/33608238 (ver. 0)
    《在中国共产党第七次代表大会上的讲话》 - JoJo王颀
    task add: an hour ago, last watch: None, last change None, next watch: an hour ago 
Task.run fetch page done
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/《在中国共产党第七次代表大会上的讲话》 - JoJo王颀的专栏 马克思的记事本.md done
Watcher.watch page task done: <Task #2501068684536>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Watcher.remember add and commit save pages 123
Task.run page request: <Task 

Task.run fetch page done
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/自省文其二 - JoJo王颀的专栏 马克思的记事本.md done
Watcher.watch page task done: <Task #2501068710632>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Task.run page request: <Task #2501068710576> https://zhuanlan.zhihu.com/p/26151013 (ver. 0)
    自省文其一 - JoJo王颀
    task add: an hour ago, last watch: None, last change None, next watch: an hour ago 
Task.run fetch page done
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/自省文其一 - JoJo王颀的专栏 马克思的记事本.md done
Watcher.watch page task done: <Task #2501068710576>


done, save_config_yaml "D:\Coding\TheNorthRemembers\test\人文主义谈话录/.task.yaml" done
Task.run page request: <Task #2501068710968> https://zhuanlan.zhihu.com/p/26147974 (ver. 0)
    （七）资本论（上） - JoJo王颀
    task add: an hour ago, last watch: None, last change None, next watch: an hour ago 
Task.run fetch page done
write D:\Coding\TheNorthRemembers\test\人文主义谈话录/（七）资本论（上） - JoJo王颀的专栏 马克思的记事本.md done
Watc

In [ ]:
w.tasks[2].should_fetch

### Fetch comments MaxRetryError

In [ ]:
Task.run page request: <Task #2500977370840> https://zhuanlan.zhihu.com/p/37509962 (ver. 0)
    毛泽东的亲民本色是他价值观的集中体现 - 林之辛
    task add: 14 minutes ago, last watch: None, last change None, next watch: 14 minutes ago 
---------------------------------------------------------------------------
MaxRetryError                             Traceback (most recent call last)
c:\programdata\anaconda3\lib\site-packages\requests\adapters.py in send(self, request, stream, timeout, verify, cert, proxies)
    439                     retries=self.max_retries,
--> 440                     timeout=timeout
    441                 )

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    731                 release_conn=release_conn,
--> 732                 body_pos=body_pos, **response_kw)
    733 

c:\programdata\anaconda3\lib\site-packages\urllib3\connectionpool.py in urlopen(self, method, url, body, headers, retries, redirect, assert_same_host, timeout, pool_timeout, release_conn, chunked, body_pos, **response_kw)
    711             try:
--> 712                 retries = retries.increment(method, url, response=response, _pool=self)
    713             except MaxRetryError:

c:\programdata\anaconda3\lib\site-packages\urllib3\util\retry.py in increment(self, method, url, response, error, _pool, _stacktrace)
    387         if new_retry.is_exhausted():
--> 388             raise MaxRetryError(_pool, url, error or ResponseError(cause))
    389 

MaxRetryError: HTTPSConnectionPool(host='api.zhihu.com', port=443): Max retries exceeded with url: /articles/37509962/comments (Caused by ResponseError('too many 403 error responses',))

During handling of the above exception, another exception occurred:

RetryError                                Traceback (most recent call last)
<ipython-input-11-2329411cf8e6> in <module>()
----> 1 w.watch()

D:\Coding\TheNorthRemembers\watcher.py in watch(self)
    580     for i, task in enumerate(page_tasks_queue, 1):
    581       # log('Watcher.watch page task: {}'.format(task))
--> 582       page_json = task.run()
    583       is_modified = task.save(page_json)
    584       task.schedule(is_modified=is_modified)  # is_modified = 跟上次存储的页面有区别

D:\Coding\TheNorthRemembers\watcher.py in run(self)
    231       log('Task.run page request: {}'.format(str(self)))
    232       fetcher = Fetcher.create(fetcher_option=self.option)
--> 233       data_json = fetcher.request()
    234       log('Task.run fetch page done')
    235       return data_json

D:\Coding\TheNorthRemembers\fetcher.py in request(self)
     46     url_type = parse_type(self.url)
     47     method = getattr(self, 'request_' + str(url_type).split('.')[-1])
---> 48     return method()
     49 
     50 

D:\Coding\TheNorthRemembers\fetcher.py in request_ZhihuColumnPage(self)
     71 
     72   def request_ZhihuColumnPage(self):
---> 73     data = fetch_zhihu_article(self.url)
     74     return data
     75 

D:\Coding\TheNorthRemembers\fetcher_api\zhihu.py in fetch_zhihu_article(article)
    558   url = zhihu_article_url(article)
    559   # log(list(article.comments))
--> 560   conversations = get_valuable_conversations(article.comments, limit=10)
    561 
    562 

D:\Coding\TheNorthRemembers\fetcher_api\zhihu.py in get_valuable_conversations(comments, limit)
    214   #     selected_ids.add(c.id)
    215 
--> 216   conversations = get_all_conversations(comments)
    217   sum_vote_count = lambda conversation: sum(comment.vote_count for comment in conversation)
    218   valuable_conversations = sorted(conversations, key=sum_vote_count, reverse=True)[:limit]

D:\Coding\TheNorthRemembers\fetcher_api\zhihu.py in get_all_conversations(comments)
    238   '''
    239   result = []
--> 240   for comment in comments:
    241     if not comment.reply_to:  # 直接评论, 开启新的会话
    242       result.append([comment])

c:\programdata\anaconda3\lib\site-packages\zhihu_oauth\zhcls\generator.py in __next__(self)
    187         while obj is None:
    188             try:
--> 189                 obj = self[self._index]
    190             except IndexError:
    191                 self._index = 0

c:\programdata\anaconda3\lib\site-packages\zhihu_oauth\zhcls\generator.py in __getitem__(self, item)
    159         while item >= self._up:
    160             if self._next_url is not None:
--> 161                 self._fetch_more()
    162             else:
    163                 raise IndexError('list index out of range')

c:\programdata\anaconda3\lib\site-packages\zhihu_oauth\zhcls\generator.py in _fetch_more(self)
     78             del params['offset']
     79 
---> 80         res = self._session.get(self._next_url, params=params)
     81         try:
     82             json_dict = res.json()

c:\programdata\anaconda3\lib\site-packages\requests\sessions.py in get(self, url, **kwargs)
    519 
    520         kwargs.setdefault('allow_redirects', True)
--> 521         return self.request('GET', url, **kwargs)
    522 
    523     def options(self, url, **kwargs):

c:\programdata\anaconda3\lib\site-packages\requests\sessions.py in request(self, method, url, params, data, headers, cookies, files, auth, timeout, allow_redirects, proxies, hooks, stream, verify, cert, json)
    506         }
    507         send_kwargs.update(settings)
--> 508         resp = self.send(prep, **send_kwargs)
    509 
    510         return resp

c:\programdata\anaconda3\lib\site-packages\requests\sessions.py in send(self, request, **kwargs)
    616 
    617         # Send the request
--> 618         r = adapter.send(request, **kwargs)
    619 
    620         # Total elapsed time of the request (approximately)

c:\programdata\anaconda3\lib\site-packages\requests\adapters.py in send(self, request, stream, timeout, verify, cert, proxies)
    497 
    498             if isinstance(e.reason, ResponseError):
--> 499                 raise RetryError(e, request=request)
    500 
    501             if isinstance(e.reason, _ProxyError):

RetryError: HTTPSConnectionPool(host='api.zhihu.com', port=443): Max retries exceeded with url: /articles/37509962/comments (Caused by ResponseError('too many 403 error responses',))

### generate token

In [ ]:
from tools import generate_zhihu_token
generate_zhihu_token()

In [ ]:
from fetcher_api import zhihu

In [ ]:
client = zhihu.client

In [ ]:
c = client.collection(133030984)
int("133030984")

In [14]:
from zhihu_oauth import ZhihuClient

TOKEN_FILE = 'token.pkl'
client = ZhihuClient()
client.load_token(TOKEN_FILE)


In [15]:
page = client.from_url('https://zhuanlan.zhihu.com/p/37509962')

In [60]:
from fetcher_api.zhihu import fetch_zhihu_article

In [61]:
fetch_zhihu_article('https://zhuanlan.zhihu.com/p/37509962')

{'comments': '错误: ForbiddenError - 106 - 评论已关闭',
 'content': '毛泽东的机要秘书谢静宜在她的回忆录里披露了这么一个小故事：\n\n《有一次，列车行进在北国大地，田地里生长着即将成熟的麦子。突然，一位肩上挑担的农妇映入眼帘。她好象是往地里送饭后返回村庄。她一边走，一边用手一把接一把地将麦穗上的麦粒往口袋里装。我惊讶地对主席说；“您看，偷麦子的。”主席忙低声说，“别出声，别出声！”他生怕伤害了那位妇女的自尊心。毛主席一直望着那位妇女的身影消失在窗外，才恍然所失地自言自话道：“看来，农民的粮食还是不够吃啊!不然，她何苦去抓那几把麦粒呢?”那天吃饭时，我注意到主席神色严肃，沉默不语，大概仍在想着那位农妇的事。》\n\n那个时候，田地及其中生长的庄稼都是属于集体的，那个农妇的行为或许会被视为“偷窃集体财物”，可是毛泽东是什么反应呢？可能出乎许多人的意料之外，他不仅没有一丝责备之意，反而流露出深深的同情，甚至还不让人发声惊动以免伤害了那个农妇的自尊心。为什么呢？毛泽东从那个农妇的举动所看到的是普通农民的艰难困苦，而且作为这个国家的领导人，他为自己的国民仍然生活困苦而自责，由此陷入沉重的深思之中，长久不能自拔。这种小到不会上书的小事情，其实是最能反映一个人的真实情感。毛泽东的反应直接出自内心，毫无做作的可能，让人无法不为他那颗与普通民众紧贴着的心所感动。\n\n文革中小学教师李庆霖写信给毛泽东反映知青的真实情况，也有人指责这封信“里头有刺”，大有追究其反对上山下乡之意，可毛泽东却说“我看了相当好啊，就是要有刺呢！我摆在这里几个月，经常看，看了三遍半”。有报道说，毛泽东看这封信读到伤心处，连眼泪也流出来了。毛泽东亲笔回信李庆霖，并“寄上三百元，聊补无米之炊。”日理万机的毛泽东对底层百姓的一封诉苦信竟如此看重，简直到了牵肠挂肚的地步！\n\n细节虽小，却往往是一个人真实思想与情感的自然反映。毛泽东是一个不屑于隐瞒自己政治观点的人，也是一个不掩饰自己情感，敢笑敢骂的性情中人。由于其特殊身份，他的一举一动，喜怒哀乐，爱谁恨谁，喜欢什么，厌恶什么，都毫无掩饰地映在身边人的眼里。我们不止一次地从毛泽东身边人听到他那些爱憎分明的“小故事”：他看不得穷苦人受难而多次掉泪，也看不得强势的人欺负弱者，对欺压百姓的行为嫉恶如仇，甚至看戏看到法

In [ ]:
import os
os.path.basename('D:\Coding\TheNorthRemembers\page.py')

In [ ]:
os.path.__file__

In [ ]:
import wechatsogou

In [ ]:
ws_api = wechatsogou.WechatSogouAPI()

In [ ]:
ws_api.get_gzh_info('南航青年志愿者')

In [ ]:
list(ws_api.search_article('马前卒工作室'))